In [1]:
import __init__ # set up all the correct referenses
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.neighbors import NearestNeighbors, KDTree, kneighbors_graph
from sklearn.metrics import pairwise_distances
from upsetplot import UpSet, from_indicators

import datetime

/exports/reum/nsteinz/lumc_torch/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/exports/reum/nsteinz/lumc_torch/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:


subproject_name = "trajectories"

wd = "../../../"
project_path = os.path.realpath(wd)
print(project_path)

raw_data_dir = project_path+"/raw_data/"
print(raw_data_dir)


data_dir = project_path+f"/{subproject_name}"+"/data/"
print(data_dir)

results_dir = project_path+f"/{subproject_name}"+"/output/"
print(results_dir)


/exports/reum/nsteinz/Project_trajectories
/exports/reum/nsteinz/Project_trajectories/raw_data/
/exports/reum/nsteinz/Project_trajectories/trajectories/data/
/exports/reum/nsteinz/Project_trajectories/trajectories/output/


In [3]:
str(datetime.date.today())

'2024-06-28'

In [4]:
raw = pd.read_csv(raw_data_dir+"RA-MAP_data_clinical.csv")
raw = raw.replace("Yes",True).replace("No", False)
drop_cols = raw.columns[raw.columns.str.contains("Date of FBC")]
raw = raw.drop(columns = drop_cols, errors="ignore")


filtered = pd.read_csv(raw_data_dir +"RA-MAP_data_clinical_filter.csv", sep=";")
filtered = raw[filtered.columns]
filtered = filtered.loc[raw.loc[:,filtered.columns[filtered.columns.str.contains("Study")][0]]== 'TACERA']

/tmp/ipykernel_1842595/2058410687.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw = raw.replace("Yes",True).replace("No", False)


In [5]:
raw = raw.loc[raw.loc[:,raw.columns[raw.columns.str.contains("Study")][0]]== 'TACERA']


In [6]:
for i in raw.columns:
   print(i)

Unnamed: 0
Subject ID
\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA Unit\
\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA Value\
\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\
\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\RF Unit\
\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\RF Value\
\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\Rheum Factor positive\
\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA Unit SCR\
\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA Value SCR\
\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA positive SCR\
\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\RF Unit SCR\
\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\RF Value SCR\
\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\Rheum Factor positive SCR\
\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA Unit 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA Value 00BL\
\RAMAP\RAMAP\Clinic

In [7]:
subjects = raw.columns[raw.columns.str.contains("Subjects")]
subjects

Index(['\RAMAP\RAMAP\Subjects\Demographics\Age\',
       '\RAMAP\RAMAP\Subjects\Demographics\Height\',
       '\RAMAP\RAMAP\Subjects\Demographics\Race\',
       '\RAMAP\RAMAP\Subjects\Demographics\Sex\',
       '\RAMAP\RAMAP\Subjects\Demographics\Weight\',
       '\RAMAP\RAMAP\Subjects\Medical History\Alcohol History\Consumption frequency\',
       '\RAMAP\RAMAP\Subjects\Medical History\Alcohol History\Drinks alcohol\',
       '\RAMAP\RAMAP\Subjects\Medical History\Alcohol History\Units of alcohol consumption per week (if over 20)\',
       '\RAMAP\RAMAP\Subjects\Medical History\Alcohol History\Units of alcohol consumption per week Range\',
       '\RAMAP\RAMAP\Subjects\Medical History\Alcohol History\Units of alcohol consumption per week\',
       '\RAMAP\RAMAP\Subjects\Medical History\Any intramuscular steroid injections\',
       '\RAMAP\RAMAP\Subjects\Medical History\Corticosteroid treatment for current episode\',
       '\RAMAP\RAMAP\Subjects\Medical History\DMARD and TNF inhibito

In [8]:
Medication = raw.columns[raw.columns.str.contains("Medication")]
Medication
Clinical = raw.columns[raw.columns.str.contains("Clinical Data")]
Clinical
xray = raw.columns[raw.columns.str.contains("XRAY")]
xray

Index(['\RAMAP\RAMAP\XRAY\Annualised Progression\',
       '\RAMAP\RAMAP\XRAY\Erosive\', '\RAMAP\RAMAP\XRAY\Final Score\',
       '\RAMAP\RAMAP\XRAY\Initial Score\',
       '\RAMAP\RAMAP\XRAY\Only 6 Month X-Rays\',
       '\RAMAP\RAMAP\XRAY\Unadjusted Progression\'],
      dtype='object')

In [9]:
steriods = raw.columns[raw.columns.str.contains("Steroid Injections")]
steriods

Index(['\RAMAP\RAMAP\Steroid Injections\03 Months\Dexamethasone 03M\',
       '\RAMAP\RAMAP\Steroid Injections\03 Months\Methylprednisolone 03M\',
       '\RAMAP\RAMAP\Steroid Injections\03 Months\Methylprednisolone Acetate 03M\',
       '\RAMAP\RAMAP\Steroid Injections\03 Months\Triamcinolone (Kenalog) 03M\',
       '\RAMAP\RAMAP\Steroid Injections\06 Months\Dexamethasone 06M\',
       '\RAMAP\RAMAP\Steroid Injections\06 Months\Methylprednisolone 06M\',
       '\RAMAP\RAMAP\Steroid Injections\06 Months\Methylprednisolone Acetate 06M\',
       '\RAMAP\RAMAP\Steroid Injections\06 Months\Triamcinolone (Kenalog) 06M\',
       '\RAMAP\RAMAP\Steroid Injections\09 Months\Methylprednisolone 09M\',
       '\RAMAP\RAMAP\Steroid Injections\09 Months\Methylprednisolone Acetate 09M\',
       '\RAMAP\RAMAP\Steroid Injections\09 Months\Triamcinolone (Kenalog) 09M\',
       '\RAMAP\RAMAP\Steroid Injections\12 Months\Dexamethasone 12M\',
       '\RAMAP\RAMAP\Steroid Injections\12 Months\Methylpredniso

In [10]:
for i in filtered.columns:
   print(i)

Subject ID
\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\
\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\Rheum Factor positive\
\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA positive SCR\
\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\Rheum Factor positive SCR\
\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA positive 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Basophils 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Eosinophils 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\HB 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Lymphocytes 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Monocytes 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Neutrophils 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\PLT 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\WBC 00BL\
\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\CRP Value (mg/L) 00BL\
\RAMAP\RAMAP\Clinical D

In [11]:
z = raw.loc[:,raw.columns.str.contains("HB ")]
z[z<20].dropna()

,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\HB 00BL\,\RAMAP\RAMAP\Clinical Data\06 Months\Blood count\HB 06M\,\RAMAP\RAMAP\Clinical Data\12 Months\Blood count\HB 12M\,\RAMAP\RAMAP\Clinical Data\18 Months\Blood count\HB 18M\
83,12.1,11.2,9.9,8.6
88,12.0,11.5,12.6,13.6
89,14.3,13.4,13.3,13.6


In [12]:
#remove = [1,3,4,6,7,9,10,12,13,15,29,30,31,33,42,43,45,46,47, 50, 51,52,54, 59, 63, 64,65]

for nr, i in enumerate(filtered.columns):
    print(f'{nr} "{i}"')

filtered.index = filtered["Subject ID"]
person_info = filtered.iloc[:,-8:].copy()
medication = filtered.loc[:,list(Medication)].copy()
steriods_usage = filtered.loc[:,list(steriods)].copy()
xray_data = raw.loc[:,list(xray)].copy()

subjects_data = raw.loc[:,list(subjects)].copy()

medication.index = raw["Subject ID"]
steriods_usage.index = raw["Subject ID"]
xray_data.index = raw["Subject ID"]
subjects_data.index = raw["Subject ID"]


acpa_screen = filtered.iloc[:,:5].copy()
lab_data = filtered.iloc[:,5:82].copy()

0 "Subject ID"
1 "\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\"
2 "\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\Rheum Factor positive\"
3 "\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA positive SCR\"
4 "\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\Rheum Factor positive SCR\"
5 "\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA positive 00BL\"
6 "\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Basophils 00BL\"
7 "\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Eosinophils 00BL\"
8 "\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\HB 00BL\"
9 "\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Lymphocytes 00BL\"
10 "\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Monocytes 00BL\"
11 "\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Neutrophils 00BL\"
12 "\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\PLT 00BL\"
13 "\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\WBC 00BL\"
14 "\RAMAP\RAMAP\Clinical Data\00 Baseli

In [13]:
additional_info = xray_data.merge(subjects_data, left_index=True, right_index=True)
additional_info

,\RAMAP\RAMAP\XRAY\Annualised Progression\,\RAMAP\RAMAP\XRAY\Erosive\,\RAMAP\RAMAP\XRAY\Final Score\,\RAMAP\RAMAP\XRAY\Initial Score\,\RAMAP\RAMAP\XRAY\Only 6 Month X-Rays\,\RAMAP\RAMAP\XRAY\Unadjusted Progression\,\RAMAP\RAMAP\Subjects\Demographics\Age\,\RAMAP\RAMAP\Subjects\Demographics\Height\,\RAMAP\RAMAP\Subjects\Demographics\Race\,\RAMAP\RAMAP\Subjects\Demographics\Sex\,...,\RAMAP\RAMAP\Subjects\Medical History\Date of Assessment\,\RAMAP\RAMAP\Subjects\Medical History\Date of symptom onset\,\RAMAP\RAMAP\Subjects\Medical History\Smoking History\If previous or current smoker Other Type\,\RAMAP\RAMAP\Subjects\Medical History\Smoking History\If previous or current smoker Type\,\RAMAP\RAMAP\Subjects\Medical History\Smoking History\If previous smoker date stopped smoking\,\RAMAP\RAMAP\Subjects\Medical History\Smoking History\Number of cigarettes per day\,\RAMAP\RAMAP\Subjects\Medical History\Smoking History\Number of years smoked\,\RAMAP\RAMAP\Subjects\Medical History\Smoking History\Smoking status\,\RAMAP\RAMAP\Subjects\Status\Date of Report\,\RAMAP\RAMAP\Subjects\Status\Enrollment\
Subject ID,,,,,,,,,,,,,,,,,,,,,
0A0B0E97239BE7AB5F473C8C046A69142C2213BC4ABADC3FA29EF7A0C75184E3,NaN,NaN,NaN,NaN,NaN,NaN,26,180.0,White- British,Male,...,9/6/2013,3/31/2013,NaN,NaN,NaN,NaN,NaN,Never Smoked,6/22/2015,Withdrawn -lost to follow up
747FC68D04F159A9D79B21F8228350946727DCD2E3C966AA4E75262FBD0D8E00,0.0,False,1.0,1.0,False,0.0,48,292.0,White- British,Female,...,8/23/2013,2/23/2013,NaN,Cigarrette,NaN,5,30.0,Current Smoker,4/16/2015,Completed
F7347C9F21B34DCE309FEDEFE6C3C8C4FFDA7B339BEBCB3ABED2BB744D4D1AD7,1.0,True,11.0,11.0,False,1.0,51,179.0,White- British,Male,...,8/13/2013,2/13/2013,NaN,Hand-rolled tobacco,1985-1-1,6,4.0,Previous Smoker,2/20/2015,Completed
87F62B63D5BAB1DE4A7C3955B6C0D81C9B13D1E48EF10ACBFAE2D8A56DDF6D38,NaN,NaN,NaN,NaN,NaN,NaN,30,163.0,White- British,Female,...,8/6/2013,3/1/2013,NaN,NaN,NaN,NaN,NaN,Never Smoked,10/7/2014,Withdrawn
1D64D145CB205919CFE0070487BC628EFA2D08E7AA7AB3FF2F6E8DBE56361E94,0.0,False,0.0,0.0,False,0.0,28,165.0,Asian or Asian British - Bangladeshi,Female,...,7/19/2013,6/1/2013,NaN,NaN,NaN,NaN,NaN,Never Smoked,1/27/2015,Completed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70F4E6F8F64BF8024116573140F01D393038C65E27CE76A7405EBF08F17FC6E1,1.0,False,5.0,4.0,False,1.0,66,173.0,White- British,Male,...,3/5/2014,1/11/2014,NaN,Hand-rolled tobacco,NaN,4,47.0,Current Smoker,2015-17-10,Completed
A029055653371689A0FAF774AB7B9B07FCCB0671A6030FCB1549FBEDBA29F2C7,2.0,False,14.0,12.0,False,2.0,69,160.0,White- British,Female,...,2/12/2014,10/1/2013,NaN,Cigarrette,1964-1-1,1,3.0,Previous Smoker,2015-25-11,Completed
9BFE0604CD4F53CBF89EDF95353DD57D4EE442829DABA3D7C39DC13F0C3E6345,0.0,False,0.0,0.0,False,0.0,44,166.0,White- British,Female,...,2/5/2014,2013-26-11,NaN,NaN,NaN,NaN,NaN,Never Smoked,2015-14-10,Completed


In [14]:
raw.loc[:,raw.columns[raw.columns.str.contains("positive SCR")]]

,\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA positive SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\Rheum Factor positive SCR\
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
...,...,...
265,NaN,True
266,False,True
267,True,True
268,True,False


In [15]:
medication

,\RAMAP\RAMAP\Medication\00 Baseline\Folic Acid 00BL\,\RAMAP\RAMAP\Medication\00 Baseline\Hydroxychloroquine 00BL\,\RAMAP\RAMAP\Medication\00 Baseline\Methotrexate 00BL\,\RAMAP\RAMAP\Medication\00 Baseline\Prednisolone 00BL\,\RAMAP\RAMAP\Medication\00 Baseline\Sulfasalazine 00BL\,\RAMAP\RAMAP\Medication\03 Months\Folic Acid 03M\,\RAMAP\RAMAP\Medication\03 Months\Hydroxychloroquine 03M\,\RAMAP\RAMAP\Medication\03 Months\Leflunomide 03M\,\RAMAP\RAMAP\Medication\03 Months\Methotrexate 03M\,\RAMAP\RAMAP\Medication\03 Months\Prednisolone 03M\,...,\RAMAP\RAMAP\Medication\15 Months\Tocilizumab 15M\,\RAMAP\RAMAP\Medication\18 Months\Adalimumab 18M\,\RAMAP\RAMAP\Medication\18 Months\Certolizumab 18M\,\RAMAP\RAMAP\Medication\18 Months\Etanercept 18M\,\RAMAP\RAMAP\Medication\18 Months\Folic Acid 18M\,\RAMAP\RAMAP\Medication\18 Months\Hydroxychloroquine 18M\,\RAMAP\RAMAP\Medication\18 Months\Leflunomide 18M\,\RAMAP\RAMAP\Medication\18 Months\Methotrexate 18M\,\RAMAP\RAMAP\Medication\18 Months\Sulfasalazine 18M\,\RAMAP\RAMAP\Medication\18 Months\Tocilizumab 18M\
Subject ID,,,,,,,,,,,,,,,,,,,,,
0A0B0E97239BE7AB5F473C8C046A69142C2213BC4ABADC3FA29EF7A0C75184E3,False,True,False,False,False,True,True,False,True,False,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
747FC68D04F159A9D79B21F8228350946727DCD2E3C966AA4E75262FBD0D8E00,True,True,True,False,False,True,True,False,True,False,...,False,False,True,False,True,True,False,True,False,False
F7347C9F21B34DCE309FEDEFE6C3C8C4FFDA7B339BEBCB3ABED2BB744D4D1AD7,True,True,True,False,False,True,True,False,True,False,...,False,False,False,False,True,True,False,True,False,False
87F62B63D5BAB1DE4A7C3955B6C0D81C9B13D1E48EF10ACBFAE2D8A56DDF6D38,True,False,True,False,False,True,True,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1D64D145CB205919CFE0070487BC628EFA2D08E7AA7AB3FF2F6E8DBE56361E94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,True,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70F4E6F8F64BF8024116573140F01D393038C65E27CE76A7405EBF08F17FC6E1,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,True,True,False,True,False,False
A029055653371689A0FAF774AB7B9B07FCCB0671A6030FCB1549FBEDBA29F2C7,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,True,True,False,True,False,False
9BFE0604CD4F53CBF89EDF95353DD57D4EE442829DABA3D7C39DC13F0C3E6345,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,True,False,False,True,False,False


In [16]:
acpa_screen.columns = acpa_screen.columns.str.rsplit("\\").str[-2]
acpa_screen

,NaN,ACPA positive,Rheum Factor positive,ACPA positive SCR,Rheum Factor positive SCR
Subject ID,,,,,
0A0B0E97239BE7AB5F473C8C046A69142C2213BC4ABADC3FA29EF7A0C75184E3,0A0B0E97239BE7AB5F473C8C046A69142C2213BC4ABADC...,True,True,True,True
747FC68D04F159A9D79B21F8228350946727DCD2E3C966AA4E75262FBD0D8E00,747FC68D04F159A9D79B21F8228350946727DCD2E3C966...,True,True,True,True
F7347C9F21B34DCE309FEDEFE6C3C8C4FFDA7B339BEBCB3ABED2BB744D4D1AD7,F7347C9F21B34DCE309FEDEFE6C3C8C4FFDA7B339BEBCB...,True,True,True,True
87F62B63D5BAB1DE4A7C3955B6C0D81C9B13D1E48EF10ACBFAE2D8A56DDF6D38,87F62B63D5BAB1DE4A7C3955B6C0D81C9B13D1E48EF10A...,True,True,True,True
1D64D145CB205919CFE0070487BC628EFA2D08E7AA7AB3FF2F6E8DBE56361E94,1D64D145CB205919CFE0070487BC628EFA2D08E7AA7AB3...,True,True,True,True
...,...,...,...,...,...
70F4E6F8F64BF8024116573140F01D393038C65E27CE76A7405EBF08F17FC6E1,70F4E6F8F64BF8024116573140F01D393038C65E27CE76...,True,True,NaN,True
A029055653371689A0FAF774AB7B9B07FCCB0671A6030FCB1549FBEDBA29F2C7,A029055653371689A0FAF774AB7B9B07FCCB0671A6030F...,False,True,False,True
9BFE0604CD4F53CBF89EDF95353DD57D4EE442829DABA3D7C39DC13F0C3E6345,9BFE0604CD4F53CBF89EDF95353DD57D4EE442829DABA3...,True,True,True,True


In [17]:
person_info.columns = person_info.columns.str.rsplit("\\").str[-2]


In [18]:
person_info = person_info

In [19]:
lab_data = raw.loc[:,list(Clinical)].copy()
lab_data.index = raw["Subject ID"]

lab_column_names = lab_data.columns.str.rsplit("\\").str[-2].str.rsplit(" ", n=1, expand=True)
lab_data.columns = lab_column_names
#lab_data = lab_data.drop(columns=[("GL", "12M")])
for i, data in enumerate(lab_data.columns):
    print(i, data)

usefull_columns =  [7,8,10,11] + list(range(15,184))
lab_data = lab_data.iloc[:,usefull_columns]

0 ('ACPA', 'Unit')
1 ('ACPA', 'Value')
2 ('ACPA', 'positive')
3 ('RF', 'Unit')
4 ('RF', 'Value')
5 ('Rheum Factor', 'positive')
6 ('ACPA Unit', 'SCR')
7 ('ACPA Value', 'SCR')
8 ('ACPA positive', 'SCR')
9 ('RF Unit', 'SCR')
10 ('RF Value', 'SCR')
11 ('Rheum Factor positive', 'SCR')
12 ('ACPA Unit', '00BL')
13 ('ACPA Value', '00BL')
14 ('ACPA positive', '00BL')
15 ('Basophils', '00BL')
16 ('Eosinophils', '00BL')
17 ('GL', '00BL')
18 ('Gdl', '00BL')
19 ('HB', '00BL')
20 ('Lymphocytes', '00BL')
21 ('Monocytes', '00BL')
22 ('Neutrophils', '00BL')
23 ('PLT', '00BL')
24 ('WBC', '00BL')
25 ('CRP Value (mg/L)', '00BL')
26 ('ESR Value', '00BL')
27 ('Evaluator global', '00BL')
28 ('Fatique', '00BL')
29 ('Outside 2 week window', '00BL')
30 ('Pain score', '00BL')
31 ('Patient global', '00BL')
32 ('Total 66/68 Swollen Joints', '00BL')
33 ('Total 66/68 Tender Joints', '00BL')
34 ('Total Swollen', '00BL')
35 ('Total Tender', '00BL')
36 ("Use of sign '<'", '00BL')
37 ('DAS-28 CRP', '00BL')
38 ('DAS28',

In [20]:
for i in lab_data.columns:
    
    if lab_data[i].dtype == "object":
        try:
            #print(lab_data[i].iloc[1], lab_data[i].dtype)
            lab_data[i] = lab_data[i].astype(bool)
        except:
            print(lab_data[i].iloc[1], lab_data[i].dtype)
            


In [21]:

x = lab_data.iloc[:,:4].copy()

#x.pivot_table(index=["Subject ID", "variable_1"], columns= "variable_0", values="SRC", fill_value=np.NaN)
x.columns = x.columns.droplevel(-1)+"_T"
seops_value = x
seops_value

,ACPA Value_T,ACPA positive_T,RF Value_T,Rheum Factor positive_T
Subject ID,,,,
0A0B0E97239BE7AB5F473C8C046A69142C2213BC4ABADC3FA29EF7A0C75184E3,109.0,True,49.0,True
747FC68D04F159A9D79B21F8228350946727DCD2E3C966AA4E75262FBD0D8E00,600.0,True,426.0,True
F7347C9F21B34DCE309FEDEFE6C3C8C4FFDA7B339BEBCB3ABED2BB744D4D1AD7,600.0,True,93.0,True
87F62B63D5BAB1DE4A7C3955B6C0D81C9B13D1E48EF10ACBFAE2D8A56DDF6D38,96.0,True,24.0,True
1D64D145CB205919CFE0070487BC628EFA2D08E7AA7AB3FF2F6E8DBE56361E94,340.0,True,210.0,True
...,...,...,...,...
70F4E6F8F64BF8024116573140F01D393038C65E27CE76A7405EBF08F17FC6E1,NaN,True,20.0,True
A029055653371689A0FAF774AB7B9B07FCCB0671A6030FCB1549FBEDBA29F2C7,NaN,False,21.0,True
9BFE0604CD4F53CBF89EDF95353DD57D4EE442829DABA3D7C39DC13F0C3E6345,340.0,True,479.0,True


In [22]:
lab_melt = lab_data.melt(ignore_index=False)
lab_melt.loc[lab_melt.value=="Yes", "value"] = True
lab_melt.loc[lab_melt.value=="No", "value"] = False
lab_final = lab_melt.reset_index().pivot_table(index=["Subject ID", "variable_1"], columns= "variable_0", values="value", fill_value=np.NaN)

/tmp/ipykernel_1842595/1542673022.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lab_final = lab_melt.reset_index().pivot_table(index=["Subject ID", "variable_1"], columns= "variable_0", values="value", fill_value=np.NaN)


In [23]:
lab_final

variable_0                                                     ACPA Value  \
Subject ID                                         variable_1               
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 00BL               NaN   
                                                   03M                NaN   
                                                   06M                NaN   
                                                   09M                NaN   
                                                   12M                NaN   
...                                                                   ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 09M                NaN   
                                                   12M                NaN   
                                                   15M                NaN   
                                                   18M                NaN   
                                                   SCR               24.0   

variable_0                                                     ACPA positive  \
Subject ID                                         variable_1                  
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 00BL                  NaN   
                                                   03M                   NaN   
                                                   06M                   NaN   
                                                   09M                   NaN   
                                                   12M                   NaN   
...                                                                      ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 09M                   NaN   
                                                   12M                   NaN   
                                                   15M                   NaN   
                                                   18M                   NaN   
                                                   SCR                   1.0   

variable_0                                                     Basophils  \
Subject ID                                         variable_1              
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 00BL              0.0   
                                                   03M               NaN   
                                                   06M               0.0   
                                                   09M               NaN   
                                                   12M               NaN   
...                                                                  ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 09M               NaN   
                                                   12M               0.0   
                                                   15M               NaN   
                                                   18M               0.0   
                                                   SCR               NaN   

variable_0                                                     CRP Value (mg/L)  \
Subject ID                                         variable_1                     
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 00BL                    35.0   
                                                   03M                     23.0   
                                                   06M                     12.0   
                                                   09M                      NaN   
                                                   12M                      NaN   
...                                                                         ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 09M                      4.0   
                                                   12M                      5.0   
                                                   15M                      6.0   
                                                   18M 

In [24]:
lab_final = lab_final.drop(columns=["ACPA positive", "Rheum Factor positive"])

In [25]:
medication.replace("Yes",1).replace("No",0)

,\RAMAP\RAMAP\Medication\00 Baseline\Folic Acid 00BL\,\RAMAP\RAMAP\Medication\00 Baseline\Hydroxychloroquine 00BL\,\RAMAP\RAMAP\Medication\00 Baseline\Methotrexate 00BL\,\RAMAP\RAMAP\Medication\00 Baseline\Prednisolone 00BL\,\RAMAP\RAMAP\Medication\00 Baseline\Sulfasalazine 00BL\,\RAMAP\RAMAP\Medication\03 Months\Folic Acid 03M\,\RAMAP\RAMAP\Medication\03 Months\Hydroxychloroquine 03M\,\RAMAP\RAMAP\Medication\03 Months\Leflunomide 03M\,\RAMAP\RAMAP\Medication\03 Months\Methotrexate 03M\,\RAMAP\RAMAP\Medication\03 Months\Prednisolone 03M\,...,\RAMAP\RAMAP\Medication\15 Months\Tocilizumab 15M\,\RAMAP\RAMAP\Medication\18 Months\Adalimumab 18M\,\RAMAP\RAMAP\Medication\18 Months\Certolizumab 18M\,\RAMAP\RAMAP\Medication\18 Months\Etanercept 18M\,\RAMAP\RAMAP\Medication\18 Months\Folic Acid 18M\,\RAMAP\RAMAP\Medication\18 Months\Hydroxychloroquine 18M\,\RAMAP\RAMAP\Medication\18 Months\Leflunomide 18M\,\RAMAP\RAMAP\Medication\18 Months\Methotrexate 18M\,\RAMAP\RAMAP\Medication\18 Months\Sulfasalazine 18M\,\RAMAP\RAMAP\Medication\18 Months\Tocilizumab 18M\
Subject ID,,,,,,,,,,,,,,,,,,,,,
0A0B0E97239BE7AB5F473C8C046A69142C2213BC4ABADC3FA29EF7A0C75184E3,False,True,False,False,False,True,True,False,True,False,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
747FC68D04F159A9D79B21F8228350946727DCD2E3C966AA4E75262FBD0D8E00,True,True,True,False,False,True,True,False,True,False,...,False,False,True,False,True,True,False,True,False,False
F7347C9F21B34DCE309FEDEFE6C3C8C4FFDA7B339BEBCB3ABED2BB744D4D1AD7,True,True,True,False,False,True,True,False,True,False,...,False,False,False,False,True,True,False,True,False,False
87F62B63D5BAB1DE4A7C3955B6C0D81C9B13D1E48EF10ACBFAE2D8A56DDF6D38,True,False,True,False,False,True,True,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1D64D145CB205919CFE0070487BC628EFA2D08E7AA7AB3FF2F6E8DBE56361E94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,True,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70F4E6F8F64BF8024116573140F01D393038C65E27CE76A7405EBF08F17FC6E1,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,True,True,False,True,False,False
A029055653371689A0FAF774AB7B9B07FCCB0671A6030FCB1549FBEDBA29F2C7,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,True,True,False,True,False,False
9BFE0604CD4F53CBF89EDF95353DD57D4EE442829DABA3D7C39DC13F0C3E6345,True,False,True,False,False,True,False,False,True,False,...,False,False,False,False,True,False,False,True,False,False


In [26]:
medication = filtered.iloc[:,82:-8].copy().fillna(False).replace("Yes",1).replace("No",0)
test = medication.columns.str.rsplit("\\").str[-2].str.rsplit(" ", n=1, expand=True)
medication.columns = test
med_melt = medication.melt(ignore_index=False)
med_melt
med_final = med_melt.pivot_table(index=["Subject ID", "variable_1"], columns= "variable_0", values="value", fill_value=0)


/tmp/ipykernel_1842595/1734452910.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  medication = filtered.iloc[:,82:-8].copy().fillna(False).replace("Yes",1).replace("No",0)


In [27]:
test = steriods_usage.fillna(False).copy()
test.columns =test.columns.str.rsplit("\\").str[-2].str.rsplit(" ", n=1, expand=True)
steriods_melt = test.melt(ignore_index=False)
steriods_melt
steriods_melt.value.sum()

/tmp/ipykernel_1842595/3020133455.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = steriods_usage.fillna(False).copy()


319

In [28]:


steriods_final = steriods_melt.pivot_table(index=["Subject ID", "variable_1"], columns= "variable_0", values="value", fill_value=0)
steriods_final.sum()
steriods_final

variable_0                                                     Dexamethasone  \
Subject ID                                         variable_1                  
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 03M                   0.0   
                                                   06M                   0.0   
                                                   09M                   0.0   
                                                   12M                   0.0   
                                                   15M                   0.0   
...                                                                      ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 06M                   0.0   
                                                   09M                   0.0   
                                                   12M                   0.0   
                                                   15M                   0.0   
                                                   18M                   0.0   

variable_0                                                     Methylprednisolone  \
Subject ID                                         variable_1                       
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 03M                        0.0   
                                                   06M                        0.0   
                                                   09M                        0.0   
                                                   12M                        0.0   
                                                   15M                        0.0   
...                                                                           ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 06M                        0.0   
                                                   09M                        0.0   
                                                   12M                        0.0   
                                                   15M                        0.0   
                                                   18M                        0.0   

variable_0                                                     Methylprednisolone Acetate  \
Subject ID                                         variable_1                               
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 03M                                0.0   
                                                   06M                                0.0   
                                                   09M                                0.0   
                                                   12M                                0.0   
                                                   15M                                0.0   
...                                                                                   ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 06M                                1.0   
                                                   09M                                1.0   
                                                   12M                                0.0   
                                                   15M                                0.0   
                                                   18M                                0.0   

variable_0                                                     Triamcinolone (Kenalog)  
Subject ID                                         variable_1                           
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 03M                             0.0  
                                                   06M                             0.0  
                                                   09M                             0.0  
                                                   12M                             0.0  
                                                   15M                             0.0  
...                 

In [29]:
med_final

variable_0                                                     Adalimumab  \
Subject ID                                         variable_1               
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 00BL               0.0   
                                                   03M                0.0   
                                                   06M                0.0   
                                                   09M                0.0   
                                                   12M                0.0   
...                                                                   ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 06M                0.0   
                                                   09M                0.0   
                                                   12M                0.0   
                                                   15M                0.0   
                                                   18M                0.0   

variable_0                                                     Certolizumab  \
Subject ID                                         variable_1                 
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 00BL                 0.0   
                                                   03M                  0.0   
                                                   06M                  0.0   
                                                   09M                  0.0   
                                                   12M                  0.0   
...                                                                     ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 06M                  0.0   
                                                   09M                  0.0   
                                                   12M                  0.0   
                                                   15M                  0.0   
                                                   18M                  0.0   

variable_0                                                     Dexamethasone  \
Subject ID                                         variable_1                  
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 00BL                  0.0   
                                                   03M                   0.0   
                                                   06M                   0.0   
                                                   09M                   0.0   
                                                   12M                   0.0   
...                                                                      ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 06M                   0.0   
                                                   09M                   0.0   
                                                   12M                   0.0   
                                                   15M                   0.0   
                                                   18M                   0.0   

variable_0                                                     Etanercept  \
Subject ID                                         variable_1               
0073B9C958B3CA3EED99BB233FD6AA03D42883B16AED8DF... 00BL               0.0   
                                                   03M                0.0   
                                                   06M                0.0   
                                                   09M                0.0   
                                                   12M                0.0   
...                                                                   ...   
FF9C17EC739F92CED4EDC266471525E0F50F8A81C9129B4... 06M                0.0   
                                                   09M                0.0   
                                                   12M                0.0   
                                                   15M                1.0   
     

In [30]:
timepoints = ['00 Baseline','06 Months','12 Months','18 Months']

In [34]:
test  = filtered.melt(id_vars="Subject ID")

In [35]:
test["split"] = test.variable.str.rsplit("\\").str[4:]
test["split_space"] = test.variable.str.rsplit(" ").str[-1].str.rstrip("\\")
test

,Subject ID,variable,value,split,split_space
0,0A0B0E97239BE7AB5F473C8C046A69142C2213BC4ABADC...,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibilit...,True,"[.ACPA/RF Eligibility, ACPA, ACPA positive, ]",positive
1,747FC68D04F159A9D79B21F8228350946727DCD2E3C966...,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibilit...,True,"[.ACPA/RF Eligibility, ACPA, ACPA positive, ]",positive
2,F7347C9F21B34DCE309FEDEFE6C3C8C4FFDA7B339BEBCB...,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibilit...,True,"[.ACPA/RF Eligibility, ACPA, ACPA positive, ]",positive
3,87F62B63D5BAB1DE4A7C3955B6C0D81C9B13D1E48EF10A...,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibilit...,True,"[.ACPA/RF Eligibility, ACPA, ACPA positive, ]",positive
4,1D64D145CB205919CFE0070487BC628EFA2D08E7AA7AB3...,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibilit...,True,"[.ACPA/RF Eligibility, ACPA, ACPA positive, ]",positive
...,...,...,...,...,...
43947,70F4E6F8F64BF8024116573140F01D393038C65E27CE76...,\RAMAP\RAMAP\XRAY\Final Score\,5.0,"[Final Score, ]",Score
43948,A029055653371689A0FAF774AB7B9B07FCCB0671A6030F...,\RAMAP\RAMAP\XRAY\Final Score\,14.0,"[Final Score, ]",Score
43949,9BFE0604CD4F53CBF89EDF95353DD57D4EE442829DABA3...,\RAMAP\RAMAP\XRAY\Final Score\,0.0,"[Final Score, ]",Score
43950,5687002206D198F4B471FE04F33F586E696A49CAF52B0A...,\RAMAP\RAMAP\XRAY\Final Score\,9.0,"[Final Score, ]",Score


In [33]:
test.pivot_table(index="Subject ID", columns="variable", values="value")

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
for nr, time in enumerate(timepoints):
    inter = test[test.split.str[0] == time].copy()
    inter["name"] = inter.split.str[-2].str.rsplit(" ").str[0]
    print(inter[["name", "value"]])
    #print(inter.reset_index(drop=True).pivot_table(index = "Subject ID", columns="name", values="value", aggfunc=np.sum))

In [ ]:
acpa_screen

In [ ]:
print(acpa_screen.columns)
t = acpa_screen.dropna().merge(seops_value, left_index=True, right_index=True).drop(columns = [np.NaN])
print(t.columns)
t
(t["ACPA positive"] != t["ACPA positive_T"]).sum()
t
#print((t["Rheum Factor positive_x"] == t["Rheum Factor positive_y"]).sum(), t["Rheum Factor positive_x"], t["Rheum Factor positive_y"])

In [ ]:
t["RF"]= t[["Rheum Factor positive", "Rheum Factor positive_T"]].max(axis=1) == 1
t["acpa"]= t[["ACPA positive", "ACPA positive_T"]].max(axis=1) == 1
t["acpa_value"] = t["ACPA Value_T"]
t["rf_value"] = t["RF Value_T"]
acpa_screen = t[["RF", "acpa", "acpa_value", "rf_value"]]

In [ ]:
person_info = person_info.merge(acpa_screen,left_index=True, right_index=True)
#person_info = person_info.merge(additional_info,left_index=True, right_index=True)
person_info

In [ ]:
lab_final

In [ ]:
person_info.merge(lab_final,left_index=True, right_index=True, how="left")

In [ ]:
person_info.merge(lab_final,left_index=True, right_index=True, how="left").columns

In [ ]:
final_data = person_info.merge(lab_final,left_index=True, right_index=True).merge(med_final,left_index=True, right_index=True)

In [ ]:
final_data = final_data.reset_index().rename(columns={"variable_1":"months"})

In [ ]:
final_data.months = final_data.months.str.strip("BL").str.strip("M").astype(int)
final_data["days"] = final_data.months * 30

In [ ]:
final_data

In [ ]:
#patnr = (final_data.query("WBC>0").groupby("Subject ID").count() >=2).query("Study == True").index
#final_data = final_data.query("`Subject ID` in @patnr")
final_data.columns

In [ ]:
final_data.to_csv(data_dir +f"TACERA_data_{datetime.date.today()}.csv", index=False)
data_dir +"TACERA_data.csv",
print(datetime.date.today())

In [ ]:
additional_info.to_csv(data_dir +f"TACERA_data_add_{datetime.date.today()}.csv")
data_dir +"TACERA_data.csv",
print(datetime.date.today())

In [ ]:
final_data.groupby('Subject ID').count().months.hist()
(final_data.query("WBC > 0 & HB > 20").groupby('Subject ID').count().months >=2).sum()

In [ ]:
final_data.columns

In [ ]:
final_data.plot(x="days", y="DAS28")

In [ ]:
final_data.query("WBC >0").plot(x="days", y="DAS28")

In [ ]:
(final_data.query("WBC >0").groupby("Subject ID").count().iloc[:,1] >=1).sum()

In [ ]:
(final_data.query("WBC >0").groupby("Subject ID").count().iloc[:,1] >=2).sum()

In [ ]:
(final_data.query("WBC >0").groupby("Subject ID").count().iloc[:,1] >=3).sum()

In [ ]:
(final_data.query("WBC >0").groupby("Subject ID").count().iloc[:,1] >=4).sum()

In [ ]:
final_data.columns

In [ ]:
lab_features= ['Basophils', 'CRP Value (mg/L)', 'DAS-28 CRP', 'DAS28',
       'ESR Value', 'Eosinophils', 'HB', 'Lymphocytes', 'Monocytes',
       'Neutrophils', 'PLT', 'Rheum Factor positive', 'WBC',]

In [ ]:
for feature in final_data.columns:
    print(feature)
    print(final_data.query("WBC >0 & Basophils >=0 & Eosinophils >= 0" )[feature].describe())

In [ ]:
final_data.HB * 0.0620599999999958

In [ ]:
description = {}
df = final_data.query("WBC >0 & Basophils >=0 & Eosinophils >= 0 & days == 0" )
for feature in final_data.columns[3:]:
    #print(feature, df[feature].dtype)
    if pd.api.types.is_numeric_dtype(df[feature]):
        description[feature] = (df[feature].count(),df[feature].mean(), df[feature].std())
pd.DataFrame(description, index = ["count", "mean", "std"]).T

In [ ]:
pd.DataFrame(description, index = ["count", "mean", "std"]).T.iloc[4:-1]

In [ ]:
final_data.query("days==0")["Sex"].describe()

In [ ]:
final_data.WBC.hist()